## Hybrid Retriever

pros:
- Higher Recall - Catch More Relevant Documents
- Flexible to Vocabulary Mismatch
- Better for Technical/Mixed Content/E-commerce (code, documents)

cons:
- Higher Computational Cost
- Little Tuning Complexity (alpha tunning, score distribution mismatches)
- Storage Overhead (2x-3x storage requirement)
#### note
- bm25 score rare words higher -> good for medical and scientific docs

In [1]:
import langchain
print(langchain.__version__)

1.2.0


In [2]:
import langchain
import langchain_core

print("langchain:", langchain.__version__)
print("langchain_core:", langchain_core.__version__)


langchain: 1.2.0
langchain_core: 1.2.2


In [3]:
import langchain
import inspect
print(langchain.__version__)
print(inspect.getfile(langchain))


1.2.0
g:\extracted\ML\RAG\.venv\Lib\site-packages\langchain\__init__.py


In [9]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.retrievers import BM25Retriever
from langchain_classic.retrievers import EnsembleRetriever # my version is 1.1.2 but need 1.2 version also changed to _classic
from langchain_core.documents import Document


In [13]:
!pip uninstall langchain langchain-core -y
!pip install langchain==1.2.0

Found existing installation: langchain 1.0.2
Uninstalling langchain-1.0.2:
  Successfully uninstalled langchain-1.0.2
Found existing installation: langchain-core 1.0.1
Uninstalling langchain-core-1.0.1:
  Successfully uninstalled langchain-core-1.0.1

  Attempting uninstall: langgraph-sdk

    Found existing installation: langgraph-sdk 0.2.9

    Uninstalling langgraph-sdk-0.2.9:

      Successfully uninstalled langgraph-sdk-0.2.9

   ------ --------------------------------- 1/6 [langgraph-sdk]
   ------ --------------------------------- 1/6 [langgraph-sdk]
   ------ --------------------------------- 1/6 [langgraph-sdk]
   ------ --------------------------------- 1/6 [langgraph-sdk]
   ------ --------------------------------- 1/6 [langgraph-sdk]
   ------ --------------------------------- 1/6 [langgraph-sdk]
   ------------- -------------------------- 2/6 [langchain-core]
   ------------- -------------------------- 2/6 [langchain-core]
   ------------- -------------------------- 2/6 [l

In [25]:
docs = [
    Document(page_content="هوش مصنوعی بسیاری از صنایع را متحول کرده است."),
    Document(page_content="در حوزه مراقبت‌های بهداشتی، الگوریتم‌های هوش مصنوعی اکنون می‌توانند بیماری‌ها را از تصاویر پزشکی با دقتی فراتر از متخصصان انسانی تشخیص دهند."),
    Document(page_content="این نشان‌دهنده یک پیشرفت بزرگ در پزشکی تشخیصی است."),
    Document(page_content="علم آب و هوا از طریق بهبود پیش‌بینی آب و هوا و مدل‌سازی آب و هوا از هوش مصنوعی بهره می‌برد."),
    Document(page_content="سالالانه 205 تن موز برداشت میشود")
]

In [33]:
bm25 = BM25Retriever.from_documents(documents=docs)
bm25.k = 2

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
dense_vectorstore = FAISS.from_documents(docs, embedding_model)
dense_retriever = dense_vectorstore.as_retriever()

sparse_retriever=BM25Retriever.from_documents(docs)
sparse_retriever.k=2 # top k documents to retriever

# Combine with Ensemble Retriever
hybrid_retriever=EnsembleRetriever(
    retrievers=[dense_retriever,sparse_retriever],
    weights=[0.7,0.3]
)


In [40]:
# Test
docstest = hybrid_retriever.invoke(
    "هوش مصنوعی در موز وجود دارد"
    
)


In [42]:

for d in docstest[:-3]:
    print(d)


page_content='در حوزه مراقبت‌های بهداشتی، الگوریتم‌های هوش مصنوعی اکنون می‌توانند بیماری‌ها را از تصاویر پزشکی با دقتی فراتر از متخصصان انسانی تشخیص دهند.'
page_content='هوش مصنوعی بسیاری از صنایع را متحول کرده است.'


### RAG Pipeline with hybrid retriever

In [47]:
# chain creation
from langchain_classic.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain.chat_models import init_chat_model
from langchain_groq import ChatGroq
from dotenv import load_dotenv


In [45]:
system_prompt = """"تو یک دستیار وبسایت هستی که باید در قالب نهاتا 40 کلمه به کاربر بر اساس اطلاعات سایت پاسخ بدهی.
اگر پاسخ سوال کاربر را نمی  دانستی بگو متاسفم ولی در این مورد نمیتوانم کمکتان کنم.

اطلاعات سایت : {context}
"""
template = ChatPromptTemplate.from_messages(
            [
                ("system", system_prompt),
                ("human", "{input}"),
            ]
        )
template

ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='"تو یک دستیار وبسایت هستی که باید در قالب نهاتا 40 کلمه به کاربر بر اساس اطلاعات سایت پاسخ بدهی.\nاگر پاسخ سوال کاربر را نمی  دانستی بگو متاسفم ولی در این مورد نمیتوانم کمکتان کنم.\n\nاطلاعات سایت : {context}\n'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])

In [48]:
import os
load_dotenv()
os.environ["GROQ_API"]=os.getenv("GROQ_API")
groq_llm = ChatGroq(model="llama-3.1-8b-instant",api_key=os.getenv("GROQ_API"))

In [49]:
template_chain = create_stuff_documents_chain(groq_llm,template)
rag_chain = create_retrieval_chain(hybrid_retriever,template_chain)

In [55]:
response = rag_chain.invoke({"input":"هوش مصنوعی در پزشکی وجود دارد"})
print("Answer:\n", response["answer"])
print("\nSource Documents:")
for i, doc in enumerate(response["context"]):
    print(f"\nDoc {i+1}: {doc.page_content}")

Answer:
 بله، هوش مصنوعی در پزشکی به صورت پیشرفته‌ای در حال توسعه است. به‌ویژه در تشخیص بیماری‌ها از تصاویر پزشکی، می‌تواند دقتی فراتر از متخصصان انسانی را داشته باشد.

Source Documents:

Doc 1: در حوزه مراقبت‌های بهداشتی، الگوریتم‌های هوش مصنوعی اکنون می‌توانند بیماری‌ها را از تصاویر پزشکی با دقتی فراتر از متخصصان انسانی تشخیص دهند.

Doc 2: این نشان‌دهنده یک پیشرفت بزرگ در پزشکی تشخیصی است.

Doc 3: هوش مصنوعی بسیاری از صنایع را متحول کرده است.

Doc 4: علم آب و هوا از طریق بهبود پیش‌بینی آب و هوا و مدل‌سازی آب و هوا از هوش مصنوعی بهره می‌برد.
